In [2]:
from queue import LifoQueue

SIZE = 8

# DO NOT CHANGE THE ORDER OF THE ITEMS!
DIRECTIONS = [(1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1), (0, 1)]
DIRECTIONS_OFFSETS = [SIZE + 1, SIZE, SIZE - 1, -1, -(SIZE + 1), -SIZE, -(SIZE - 1), 1]


def calculate_move(me: int, move: tuple[int, int], board: tuple[int, int]) -> tuple[int, int]:
    """
    `me` should be 0 or 1.
    """
    taken_places, colors = board

    row, col = move
    move_position_bit = 1 << (row * SIZE + col)

    colors_flip = 0

    for (d_row, d_col), bit_offset in zip(DIRECTIONS, DIRECTIONS_OFFSETS):
        row, col = move

        bit = move_position_bit
        flip_n = 0

        should_affect = False

        while True:
            row += d_row
            col += d_col

            if bit_offset < 0:
                bit >>= -bit_offset
            else:
                bit <<= bit_offset

            if row >= SIZE or col >= SIZE or not (taken_places & bit):
                break

            if ((colors & bit) > 0) is (me > 0):  # a connection was made
                should_affect = True
                break

            flip_n |= bit

        if should_affect:
            colors_flip ^= flip_n

    if me:
        colors_flip ^= move_position_bit

    return move_position_bit, colors_flip


def apply_move(board: tuple[int, int], move_flip: tuple[int, int]) -> tuple[int, int]:
    taken_places, colors = board
    position_flip, colors_flip = move_flip

    return taken_places ^ position_flip, colors ^ colors_flip


def translate_to_bin(board: list[list[int]]) -> tuple[int, int]:
    """
    2 values -> 1
    1 values -> 0
    """
    taken_places, colors = 0, 0
    bit = 1

    for row in range(SIZE):
        for col in range(SIZE):
            value = board[row][col]
            if value != 0:
                taken_places |= bit

            if value == 2:
                colors |= bit

            bit <<= 1

    return taken_places, colors


def preview_board(board: tuple[int, int]):
    taken_places, colors = board

    bit = 1
    board_repr = ""
    for i in range(SIZE):
        for j in range(SIZE):
            is_place_taken = taken_places & bit
            if not is_place_taken:
                board_repr += "."
            else:
                board_repr += "1" if colors & bit else "0"
            bit <<= 1
        board_repr += "\n"
    return board_repr


STARTING_BOARD = [[0] * SIZE for _ in range(SIZE)]
STARTING_BOARD[SIZE // 2 - 1][SIZE // 2 - 1] = 1
STARTING_BOARD[SIZE // 2][SIZE // 2] = 1
STARTING_BOARD[SIZE // 2 - 1][SIZE // 2] = 2
STARTING_BOARD[SIZE // 2][SIZE // 2 - 1] = 2

STARTING_BIN_BOARD = translate_to_bin(STARTING_BOARD)


def play():
    bin_board = STARTING_BIN_BOARD
    moves: LifoQueue = LifoQueue()

    me = 0
    while True:
        print(preview_board(bin_board))

        s = input(">>>")
        if s == "q":  # quit game
            break

        if s == "r":  # reverse last move
            move = moves.get()
        else:  # make a move: row then column separated with space
            row_s, col_s = s.split(" ")
            row, col = int(row_s), int(col_s)
            move = calculate_move(me, (row, col), bin_board)
            moves.put(move)

        bin_board = apply_move(bin_board, move)
        me = 1 - me


if __name__ == '_main_':
    play()